<a href="https://colab.research.google.com/github/prrmzz/RAG-for-Iranian-High-School-Biology-Textbook/blob/main/HighschoolBiologyRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain transformers sentence-transformers faiss-cpu pdfminer.six pypdf

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
PDF_FOLDER = "/content/drive/MyDrive/IranianSchoolsBiologyBooks"

# Load all PDFs
all_docs = []
for file in os.listdir(PDF_FOLDER):
    if file.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(PDF_FOLDER, file))
        docs = loader.load()
        all_docs.extend(docs)

print(f"Loaded {len(all_docs)} PDF documents.")

# Split into smaller chunks for embeddings
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
texts = text_splitter.split_documents(all_docs)
print(f"Total chunks: {len(texts)}")


Loaded 424 PDF documents.
Total chunks: 913


In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embedding_model)

# Optional: save FAISS locally
vectorstore.save_local("biology_faiss_index")


/tmp/ipython-input-646899693.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access publi

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # free & instruction-tuned

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Use HF pipeline with LangChain
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.9
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-839905569.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)


In [ ]:
query = "توضیح دهید چرخه سلولی چیست و مراحل آن را به زبان ساده بیان کنید."
result = qa(query)

print("💬 Answer:\n", result['result'])
print("\n📚 Sources:")
for doc in result['source_documents']:
    print("-", doc.metadata.get('source', 'Unknown'))


/tmp/ipython-input-573992005.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(query)


💬 Answer:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

تجرعلومان
پژوهشگرودانشمندان
یاوها
ساختارفقطشناسی،زیستدراین،
بنابراست؛بی
تجرعلوماساسِ مشاهده،اند.مشاهدهقابل
شناسی چیست؟ زیست  گفتار

زیر
د
ه
شدمشخصتیب،
تراینبهکنند.می
ید
تولدن
دربه
کادی
موی
برخه
کت
اس
یک
تحرا
رد
درای
هنده
گیرد،
شونمی
.
کنندمی
گیرنده
پیزرد
دوسرماهیچۀ
وضعیتحسهایگیرنده3شکل
ردپی
زدر

به عبارتی نقش رفتارها را در بقا و زادآوری بیشتر جانوران 
شود.  ای جانور، انجام می
رفتار بر
های کاکایی شکل
های کاکایی ب) تخم
ب)      الف)

Question: توضیح دهید چرخه سلولی چیست و مراحل آن را به زبان ساده بیان کنید.
Helpful Answer: چرخه سلولی شامل پنج مرحله است: نابرابری، حمل و بازگشت، اندازه‌گیری، فعالیت و بازگشت (اندازه‌گیری). هر مرحله این چرخه شامل تغییرات محاسبه‌ای است که در

📚 Sources:
- /content/drive/MyDrive/IranianSchoolsBiologyBooks/123.pdf
- /content/drive/MyDriv